
### Imports


In [ ]:
import os
import torch
import onnxruntime
from utils.torch_utils import (
    load_model_resnet50, 
    load_model_resnet50_minuslastlayer, 
    export_model_onnx,
    extract_embeddings_onnx,
    extract_embeddings_torch,
    save_embeddings,
    load_embeddings,
)
from utils.data_utils import load_mnist_data
from luxonis_ml.embeddings.utils.model import (
    load_model_onnx, 
    save_model_onnx, 
    extend_output_onnx,
)



### PyTorch Embeddings


In [ ]:
# Load the data
data_loader = load_mnist_data(save_path="./data/mnist", num_samples=640, batch_size=64)

# Load the PyTorch model
model = load_model_resnet50_minuslastlayer()

# Extract and save embeddings
embeddings, labels = extract_embeddings_torch(model, data_loader)
save_embeddings(embeddings, labels, "./data/")



### ONNX Embeddings


In [ ]:

# Load the ONNX model
model = load_model_resnet50()
export_model_onnx(model, model_path_out="./data/resnet50.onnx")
onnx_model = load_model_onnx(model_path="./data/resnet50.onnx")
onnx_model = extend_output_onnx(onnx_model, intermediate_tensor_name="/Flatten_output_0")
save_model_onnx(onnx_model, model_path_out="./data/resnet50-1.onnx")

# Create an ONNX Runtime session
provider = ["CUDAExecutionProvider"] if torch.cuda.is_available() and "CUDAExecutionProvider" in onnxruntime.get_available_providers() else None
ort_session = onnxruntime.InferenceSession("./data/resnet50-1.onnx", providers=provider)

# Extract and save ONNX embeddings
embeddings, labels = extract_embeddings_onnx(ort_session, data_loader, "/Flatten_output_0")
save_embeddings(embeddings, labels, "./data/")

### From LDF (Building LDF and Generating Embeddings)

In [ ]:
from luxonis_ml.data import LuxonisDataset, LuxonisLoader
from luxonis_ml.embeddings.utils.ldf import generate_embeddings

In [ ]:
BATCH_SIZE = 64
mnist_image_dir = "./data/mnist_images"

# Convert MNIST data to Luxonis ML format
def mnist_LDF_generator():
    batch_num = 0
    for batch in data_loader:
        images, labels = batch
        for i, (image, label) in enumerate(zip(images, labels)):
            img_ix = batch_num * BATCH_SIZE + i

            # Save image to disk
            # image_path = os.path.join(mnist_image_dir, f"{uuid.uuid4()}.jpg")
            image_path = os.path.join(mnist_image_dir, f"mnist_{img_ix}.jpg")
            torchvision.utils.save_image(image, image_path)

            # Create dictionary structure for Luxonis ML
            yield {
                "file": image_path,
                "class": str(label.item()),
                "type": "classification",
                "value": True,
            }
        batch_num += 1

In [ ]:
# Assuming the MNIST data is already loaded and processed into images
# Convert MNIST data to LDF and generate embeddings
dataset_name = "Mnist_LDF"
dataset = LuxonisDataset(dataset_name)
dataset.set_classes(["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"])
dataset.add(mnist_LDF_generator)  # Assuming mnist_LDF_generator is defined
dataset.make_splits()

In [ ]:
from luxonis_ml.embeddings.utils.qdrant import QdrantAPI, QdrantManager
QdrantManager("qdrant/qdrant", "qdrant_container").start_docker_qdrant()
qdrant_api = QdrantAPI("localhost", 6333)
# Create a collection
vector_size = len(embeddings[0])
qdrant_api.create_collection(
    collection_name="Mnist", 
    properties=["label","image_path"], 
    vector_size=vector_size, 
    distance="cosine"
)

In [ ]:

loader = LuxonisLoader(dataset)
ort_session = onnxruntime.InferenceSession("./data/resnet50-1.onnx", providers=provider)
emb_dict = generate_embeddings(dataset, ort_session, qdrant_api, output_layer_name="/Flatten_output_0") 


### Without LDF (Using LuxonisFileSystem)


In [ ]:

from luxonis_ml.embeddings.utils.embedding import extract_embeddings
from luxonis_ml.utils import LuxonisFileSystem

current_dir = os.getcwd()
file_info = {
    "base_path": current_dir,
    "prefix": "data/mnist_images",
    "model_path": "data/resnet50-1.onnx",
}
local_model = file_info["model_path"]

ort_session = onnxruntime.InferenceSession(local_model, providers=provider)
lfs_local = LuxonisFileSystem(file_info["base_path"])

files = []
for img in lfs_local.walk_dir(file_info["prefix"]):
    files.append(img)

embeddings, succ_ix = extract_embeddings(files, ort_session, lfs_local, preprocess_function=None, output_layer_name="/Flatten_output_0", batch_size=64)

In [ ]:
from luxonis_ml.embeddings.utils.qdrant import QdrantAPI, QdrantManager

QdrantManager("qdrant/qdrant", "qdrant_container").start_docker_qdrant()
qdrant_api = QdrantAPI("localhost", 6333)

# Create a collection
vector_size = len(embeddings[0])
qdrant_api.create_collection(
    collection_name="Mnist_LFS", 
    properties=["image_path"], 
    vector_size=vector_size, 
    distance="cosine"
)

In [ ]:
import uuid
# Insert the embeddings into the collection
ids = [str(uuid.uuid5(uuid.NAMESPACE_DNS, str(e))) for e in embeddings]
img_path_list_dict = [{"image_path": img} for i, img in enumerate(files) if i in succ_ix]
qdrant_api.insert_embeddings(ids, embeddings, img_path_list_dict, batch_size=50)